# Creating and Using Panel Data Using ArcGIS Defined Location Cubes

In [1]:
import os as OS
import arcpy as ARCPY
import SSDataObject as SSDO
import SSPanelObject as SSPO
import SSPanel as PANEL
ARCPY.overwriteOutput = True

## Example: Per Capita Incomes Relative to National Average in California (1969 - 2010)

- Create Defined Locations Cube from Repeating Shapes Feature Class
- Run Mann-Kendall Trend Statistic
- Save to NetCDF File

In [2]:
inputFC = r'../data/CA_Counties_Panel.shp'
outputCube = r'../data/CA_Panel.nc'
fullFC = OS.path.abspath(inputFC)
outputCube = OS.path.abspath(outputCube)
fullPath, fcName = OS.path.split(fullFC)
ssdo = SSDO.SSDataObject(inputFC)
uniqueIDField = "MYID"
timeField = "YEAR"
analysisField = "PCR"
panelObj = SSPO.SSPanelObject(inputFC)
requireGeometry = panelObj.ssdo.shapeType.upper() == "POLYGON"
panelObj.obtainData(uniqueIDField, "YEAR", "1 Years", fields = [analysisField],
                    requireGeometry = requireGeometry)
panelCube = PANEL.SSPanel(outputCube, panelObj = panelObj)
varName = panelObj.fieldNames[0]
panelCube.mannKendall(varName)
panelCube.close()

## Open Panel Cube From NetCDF File for Analysis 

In [3]:
panel = PANEL.SSPanel(outputCube)

## Number of Locations and Time Periods

In [4]:
print("# locations = {0}, # time periods = {1}".format(panel.numLocations, panel.numTime))

# locations = 58, # time periods = 42


## List Variables

In [5]:
print(panel.obtainVariableList())

['projection' 'time' 'x' 'y' 'lat' 'lon' 'poly_breaks' 'poly_coords'
 'time_step_ID' 'location_ID' 'MYID' 'PCR' 'PCR_TREND_ZSCORE'
 'PCR_TREND_PVALUE' 'PCR_TREND_BIN']


## View Mann-Kendall Trend Results in PANDAS Data Frame

In [6]:
import pandas as PANDAS
locations = panel.locationLabel[0]
z = panel.obtainValues('PCR_TREND_ZSCORE')
pv = panel.obtainValues('PCR_TREND_PVALUE')
d = {'PCR_TREND_ZSCORE':z, 'PCR_TREND_PVALUE':pv}
df = PANDAS.DataFrame(d, index = locations)
print(df.head())

     PCR_TREND_PVALUE  PCR_TREND_ZSCORE
158      1.552410e-03          3.164704
159      6.646548e-01         -0.433496
160      1.202954e-06         -4.855150
161      1.539366e-08         -5.657117
162      1.441587e-03         -3.186192


## Get 3D Analysis Variable 

In [7]:
data = panel.obtainValues(analysisField)
print(data.shape)

(42, 58)


## Use PySAL to Analyze LISA Markov Transitions

In [8]:
import pysal as PYSAL
w = PYSAL.open(r"../data/queen.gal").read()
lm = PYSAL.LISA_Markov(data.T, w)
print(lm.classes)

[1 2 3 4]


## View Transistion Probabilities

In [9]:
print(lm.p)

[[0.91103789 0.02800659 0.00988468 0.05107084]
 [0.07037037 0.84074074 0.08518519 0.0037037 ]
 [0.00246711 0.01726974 0.95805921 0.02220395]
 [0.09122807 0.00701754 0.09122807 0.81052632]]


In [10]:
panel.close()